In [1]:
import importlib

import indra_gpt.chat_curate.chat_curate
importlib.reload(indra_gpt.chat_curate.chat_curate)
from indra_gpt.chat_curate.chat_curate import *

import indra_gpt.chat_curate.eval
importlib.reload(indra_gpt.chat_curate.eval)
from indra_gpt.chat_curate.eval import (curation_statement_evidence_pair_comparison_json, 
                                        curation_statement_comparison_json)

import pickle
from pathlib import Path


/Users/thomaslim/miniconda3/envs/indra_gpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO: [2025-06-27 15:16:27] httpx - HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"


In [2]:
RESULTS_DIR = Path.cwd().joinpath("output/curated_statements_sample_100")
if not RESULTS_DIR.exists():
    RESULTS_DIR.mkdir(parents=True)


In [3]:
import random
import pickle
from pathlib import Path
from tqdm import tqdm
from indra.sources import indra_db_rest

# Step 1: Get all curations from the INDRA DB
print("📥 Fetching all curations from INDRA DB...")
curations = indra_db_rest.get_curations()

# Step 2: Extract unique statement hashes from curations
print("🧹 Extracting unique statement hashes...")
unique_hashes = list({c['pa_hash'] for c in curations if 'pa_hash' in c})

# Step 3: Randomly sample up to 100 curated statement hashes
sample_size = min(100, len(unique_hashes))
sampled_hashes = random.sample(unique_hashes, sample_size)

# Step 4: Fetch full statements by hash (including all evidences)
print(f"🔍 Fetching {sample_size} curated statements from INDRA DB...")
stmt_proc = indra_db_rest.get_statements_by_hash(sampled_hashes, ev_limit=100)
curated_statements = stmt_proc.statements

# Step 5: Save to pickle
output_path = RESULTS_DIR / "statements.pkl"

with open(output_path, "wb") as f:
    pickle.dump(curated_statements, f)

print(f"✅ Saved {len(curated_statements)} curated statements to: {output_path}")


INFO: [2025-06-27 15:16:28] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list
INFO: [2025-06-27 15:16:28] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-06-27 15:16:28] indra.sources.indra_db_rest.util - data: None


📥 Fetching all curations from INDRA DB...


INFO: [2025-06-27 15:16:30] indra_db_rest.query_processor - Retrieving statements that have hash -10033945991593843, -10444363735373102, -1083216214855434, -11019131917628364, -12463996681673250, -12988782223228843, -13505886640472175, -14117628316152430, -14556452501867428, -15297044990735102, -15352348890731234, -17222594879744452, -17407910080587992, -18031556530033893, -18152182327560480, -18387380848780476, -184811282650788, -18949191324265795, -19952367157294119, -20155356540515066, -20439687097155111, -2046863695546735, -2069452773758641, -2139820121868237, -22061102960387551, -22499263970759753, -22627892168184181, -22849205778179846, -23507181552861323, -24316712102315887, -25141892168903907, -25655970173329808, -25725881202629826, -26780953604978411, -28031656773092240, -29122158203077245, -29415098045592940, -32399727139857751, -33852347582520392, -3409159551394617, -34105277816332298, -34803309266443718, -35102576635162472, -35882458128194165, -35884737692158253, -493878311

🧹 Extracting unique statement hashes...
🔍 Fetching 100 curated statements from INDRA DB...
✅ Saved 92 curated statements to: /Users/thomaslim/gyorilab/indra_gpt/output/curated_statements_sample_100/statements.pkl


In [4]:
statements = pickle.load(
    open(RESULTS_DIR.joinpath("statements.pkl"), "rb")
)
statements = statements

for model in ['gpt-4o-mini', 'gpt-4o']:
    llm_curations_binary = chat_curate_stmts(statements, 
                                    n_evidence_to_curate=5,
                                    decision_threshold=0.5,
                                    binary_classification=True,
                                    #ignore_tags=['incorrect'],
                                    n_fewshot_examples=0,
                                    model=model)
    llm_curations_multiclass = chat_curate_stmts(statements,
                                    n_evidence_to_curate=5,
                                    decision_threshold=0.5,
                                    binary_classification=False,
                                    ignore_tags=['incorrect'],
                                    n_fewshot_examples=0,
                                    model=model)

    if not RESULTS_DIR.joinpath(model).exists():
        RESULTS_DIR.joinpath(model).mkdir(parents=True)
        
    with open(RESULTS_DIR / model / "chat_curation_binary_classification.json", "w") as f:
        json.dump(llm_curations_binary, f, indent=2)
    with open(RESULTS_DIR / model / "chat_curation_multiclass_classification.json", "w") as f:
        json.dump(llm_curations_multiclass, f, indent=2)


Curating statements with chat curation:   0%|          | 0/92 [00:00<?, ?it/s]15:16:46 - LiteLLM:INFO: utils.py:2931 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO: [2025-06-27 15:16:46] LiteLLM - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO: [2025-06-27 15:16:47] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
15:16:47 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO: [2025-06-27 15:16:47] LiteLLM - Wrapper: Completed Call, calling success_handler
15:16:47 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-06-27 15:16:47] LiteLLM - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
15:16:47 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-06-27 15:16:47] LiteLLM - selected model name for cost calculati

In [5]:
for model in ['gpt-4o-mini', 'gpt-4o']:
    with open(RESULTS_DIR / model / "chat_curation_binary_classification.json", "r") as f:
        chat_curation_binary_classification = json.load(f)
    with open(RESULTS_DIR / model / "chat_curation_multiclass_classification.json", "r") as f:
        chat_curation_multiclass_classification = json.load(f)

    eval_stmt_ev_pair_binary = curation_statement_evidence_pair_comparison_json(chat_curation_binary_classification)
    with open(RESULTS_DIR / model / "eval_stmt_ev_pair_chat_curation_binary_classification.json", "w") as f:
        json.dump(eval_stmt_ev_pair_binary, f, indent=2)

    eval_stmt_binary = curation_statement_comparison_json(chat_curation_binary_classification)
    with open(RESULTS_DIR / model / "eval_stmt_chat_curation_binary_classification.json", "w") as f:
        json.dump(eval_stmt_binary, f, indent=2)

    eval_stmt_ev_pair_multi = curation_statement_evidence_pair_comparison_json(chat_curation_multiclass_classification)
    with open(RESULTS_DIR / model / "eval_stmt_ev_pair_chat_curation_multiclass_classification.json", "w") as f:
        json.dump(eval_stmt_ev_pair_multi, f, indent=2)

    eval_stmt_multi = curation_statement_comparison_json(chat_curation_multiclass_classification)
    with open(RESULTS_DIR / model / "eval_stmt_chat_curation_multiclass_classification.json", "w") as f:
        json.dump(eval_stmt_multi, f, indent=2)


Processing LLM curations:   0%|          | 0/92 [00:00<?, ?it/s]INFO: [2025-06-27 16:33:57] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/-35882458128194165/5038432318346639636
INFO: [2025-06-27 16:33:57] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-06-27 16:33:57] indra.sources.indra_db_rest.util - data: None
INFO: [2025-06-27 16:33:58] indra_gpt.chat_curate.eval - Error processing evidence curation: list index out of rangeskipping this curation
INFO: [2025-06-27 16:33:58] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/-35882458128194165/-903689740779289442
INFO: [2025-06-27 16:33:58] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-06-27 16:33:58] indra.sources.indra_db_rest.util - data: None
INFO: [2025-06-27 16:33:59] indra_gpt.chat_curate.eval - Error processing evidence curation: list index out of rangeskipping this curation
INFO: [2025-06-27 16:33:59] in

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

for model in ['gpt-4o-mini', 'gpt-4o']:
    print(f"Model: {model}")
    for eval_strat in ['binary_classification', 'multiclass_classification']:
        with open(RESULTS_DIR / model / f"eval_stmt_chat_curation_{eval_strat}.json", "r") as f:
            res = json.load(f)
        preds = [res_item['llm_overall_prediction'] for res_item in res]
        labels = [res_item['indra_curation'] for res_item in res]
        acc = accuracy_score(labels, preds)
        print(f"{eval_strat} - Accuracy: {acc:.4f}")


Model: gpt-4o-mini
binary_classification - Accuracy: 0.7065
multiclass_classification - Accuracy: 0.5870
Model: gpt-4o
binary_classification - Accuracy: 0.7500
multiclass_classification - Accuracy: 0.7174


In [7]:
# from sklearn.metrics import confusion_matrix
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Extract tags
# tags = [c['tag'] for c in curation_comp_json]
# predicted_tags = [c['predicted_tag'] for c in curation_comp_json]

# # Compute accuracy
# accuracy = np.mean([t == p for t, p in zip(tags, predicted_tags)])
# accuracy_bin = np.mean([t == p or (t != 'correct' and p == 'incorrect') for t, p in zip(tags, predicted_tags)])
# print(f"Accuracy: {accuracy:.2%}")
# print(f"Binary Accuracy: {accuracy_bin:.2%}")

# # Define tag label set
# tag_set = list(set(tags).union(set(predicted_tags)))

# # Compute confusion matrix
# cm = confusion_matrix(tags, predicted_tags, labels=tag_set)
# cm_df = pd.DataFrame(cm, index=tag_set, columns=tag_set)

# # Plot
# plt.figure(figsize=(12, 10))
# sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', cbar=True, linewidths=0.5)
# plt.title("Confusion Matrix (Binary Classification - Model only predicts 'correct' or 'incorrect')")
# plt.xlabel("Predicted Label")
# plt.ylabel("True Label")
# plt.xticks(rotation=45, ha='right')
# plt.yticks(rotation=0)
# plt.tight_layout()
# plt.show()
